In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from google.colab import drive
drive.mount('/content/drive')
from glob import glob
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary
from fastai.metrics import accuracy
from fastai.vision import *
import numpy as np
!pip install fft-conv-pytorch
from fft_conv_pytorch import fft_conv, FFTConv2d

Mounted at /content/drive


In [ ]:
#Run only 1st time to download dataset into drive. Dont run if dataset is already downloaded to drive
# ! wget -P /content/drive/MyDrive https://github.com/muhammedtalo/COVID-19/archive/refs/heads/master.zip
# ! unzip -q /content/drive/MyDrive/master.zip -d /content/drive/MyDrive/Dataset

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from fastai.vision.data import normalize
path = "/content/drive/MyDrive/Dataset/brain_tumor_dataset_grayscale"
np.random.seed(41)
defaults = SimpleNamespace(cmap='L', return_fig=False, silent=False)
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(512,512), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"

In [ ]:
print (data.classes, data.c, len(data.train_ds), len(data.valid_ds))
print (data.train_ds.classes)
print (data.valid_ds.classes)

['covid-19', 'no_findings', 'pneumonia'] 3 900 225
['covid-19', 'no_findings', 'pneumonia']
['covid-19', 'no_findings', 'pneumonia']


In [ ]:


vgg_model = nn.Sequential(
    
    FFTConv2d(in_channels=1, out_channels=16, kernel_size=21, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),

    nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2), 
    
    # nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.MaxPool2d(kernel_size=2, stride=2), 
    
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Flatten(),
    nn.Linear(476288, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    # nn.Linear(4096, 4096),
    # nn.ReLU(inplace=True),
    # nn.Dropout(0.5),
    nn.Linear(512, 3)
)





In [ ]:
vgg_learn = Learner(data, vgg_model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(vgg_learn.summary())

Sequential
Layer (type)         Output Shape         Param #    Trainable 
_FFTConv             [16, 494, 494]       7,072      True      
______________________________________________________________________
ReLU                 [16, 494, 494]       0          False     
______________________________________________________________________
Conv2d               [16, 494, 494]       2,320      True      
______________________________________________________________________
ReLU                 [16, 494, 494]       0          False     
______________________________________________________________________
MaxPool2d            [16, 247, 247]       0          False     
______________________________________________________________________
Conv2d               [32, 247, 247]       4,640      True      
______________________________________________________________________
ReLU                 [32, 247, 247]       0          False     
___________________________________________________

In [ ]:
import time
start = time.time()
vgg_learn.fit_one_cycle(100, max_lr=1e-4)
end = time.time()
print(end-start)

epoch,train_loss,valid_loss,accuracy,time
0,0.956280,0.971191,0.462222,01:03
1,0.926292,0.946585,0.537778,00:52
2,0.894248,0.925212,0.613333,00:52
3,0.867873,0.842489,0.617778,00:50
4,0.834847,0.832283,0.613333,00:53
5,0.806542,0.823430,0.613333,00:49
6,0.786802,0.727203,0.622222,00:52
7,0.769704,0.699540,0.666667,00:50
8,0.725279,0.631111,0.671111,00:51
9,0.696770,0.623393,0.693333,00:51


5121.36205124855
